# WYMAGANE BIBLIOTEKI

In [70]:
!pip install opencv-python mediapipe pandas


Defaulting to user installation because normal site-packages is not writeable
  Using cached pandas-2.3.0-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (91 kB)
  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.2-py2.py3-none-any.whl.metadata (1.4 kB)
Using cached pandas-2.3.0-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (12.0 MB)
Using cached pytz-2025.2-py2.py3-none-any.whl (509 kB)
Using cached tzdata-2025.2-py2.py3-none-any.whl (347 kB)

[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python3.12 -m pip install --upgrade pip


In [63]:
import os
import sqlite3
import cv2
import mediapipe as mp
import numpy as np

DB_FILE = "video_dataset.db"
VIDEO_DIR = "/home/magda/Documents/Inżynierka/Magda/Dataset/Magdalena Machacka/videos"
TARGET_FRAMES = 30

mp_hands = mp.solutions.hands



# FUNKCJE BAZY DANYCH

In [64]:
import os
import sqlite3

file = "video_dataset.db"
directory = "/home/magda/Documents/Inżynierka/Magda/Dataset/Magdalena Machacka/videos"

# Lista katalogów
video_dirs = [dir for dir in os.listdir(directory)
              if os.path.isdir(os.path.join(directory, dir))]

try:
    # Połączenie z bazą danych
    conn = sqlite3.connect(file)
    print("Connected to database")
    created_tables = []
    # Stworzenie tabel
    tables = conn.execute("SELECT name FROM sqlite_master WHERE type='table';").fetchall()

    if not tables:
        for dir in video_dirs:
            table = f'"{dir}"'

            # Generowanie kolumn landmarków
            landmarks = ",\n".join(
                [f"landmark_{i}_{axis} REAL"
                 for i in range(1, 22) for axis in ['x', 'y', 'z']]
            )

            sql = f'''
            CREATE TABLE IF NOT EXISTS {table} (
                sample_id INTEGER,
                frame INTEGER,
                user_id TEXT,
                label TEXT,
                {landmarks}
            );
            '''
            result = conn.execute(sql)
            if result:
                created_tables += [dir]

        print(f"Created tables: {', '.join(created_tables)}")

    conn.commit()
    conn.close()

except Exception as e:
    print(f"Error connecting to database: {e}")


Connected to database
Created tables: A, A_A, B, C, C_C, CZ, CH, D, E, E_E, F, G, H, I, J, K, L, L_L, M, N, O, O_O, P, R, RZ, S, S_S, SZ, T, U, W, Z, Z_Z, Z_X, N_N, Y


# PRZETWARZANIE WIDEO

In [65]:
def interpolate_landmarks(landmarks, target_frames=30):
    landmarks = np.array(landmarks)
    count, dim = landmarks.shape

    if count == target_frames:
        return landmarks.tolist()

    original_times = np.linspace(0, 1, count)
    target_times = np.linspace(0, 1, target_frames)

    interpolated = np.zeros((target_frames, dim))
    for d in range(dim):
        interpolated[:, d] = np.interp(target_times, original_times, landmarks[:, d])

    return interpolated.tolist()


In [66]:
def extract_landmarks_from_video(path):
    cap = cv2.VideoCapture(path)
    landmarks_all = []

    with mp_hands.Hands(static_image_mode=False, max_num_hands=1, min_detection_confidence=0.5) as hands:
        while cap.isOpened():
            success, frame = cap.read()
            if not success:
                break
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            result = hands.process(frame_rgb)

            if result.multi_hand_landmarks:
                hand = result.multi_hand_landmarks[0]
                frame_landmarks = []
                for lm in hand.landmark[:21]:
                    frame_landmarks.extend([lm.x, lm.y, lm.z])
                landmarks_all.append(frame_landmarks)

    cap.release()

    if not landmarks_all:
        return None

    return interpolate_landmarks(landmarks_all, TARGET_FRAMES)


# PRZETWARZANIE CAŁEGO ZBIORU DANYCH

In [67]:
def process_all_videos():
    conn = sqlite3.connect(DB_FILE)
    cursor = conn.cursor()
    sample_id = 0
    video_dirs = [d for d in os.listdir(VIDEO_DIR) if os.path.isdir(os.path.join(VIDEO_DIR, d))]

    for dir in video_dirs:
        print(f"Processing {dir}")
        table = f'"{dir}"'
        folder_path = os.path.join(VIDEO_DIR, dir)

        for file in os.listdir(folder_path):
            print(f"Processing {file}")
            if not file.lower().endswith('.mp4'):
                continue
            filepath = os.path.join(folder_path, file)
            landmarks = extract_landmarks_from_video(filepath)
            if landmarks is None:
                print(f"Warning: no landmarks in {file}")
                continue

            user_id = os.path.splitext(file)[0]
            label = dir

            for frame_idx, frame in enumerate(landmarks):
                if len(frame) != 63:
                    print(f"Warning: invalid frame shape in {file}, frame {frame_idx}")
                    continue

                placeholders = ", ".join(["?"] * (4 + len(frame)))
                columns = ['sample_id', 'frame', 'user_id', 'label'] + \
                    [f'landmark_{i}_{axis}' for i in range(1, 22) for axis in ['x', 'y', 'z']]

                query = f'''
                INSERT INTO {table} ({", ".join(columns)})
                VALUES ({placeholders})
                '''
                cursor.execute(query, [sample_id, frame_idx, user_id, label] + frame)

            sample_id += 1

    conn.commit()
    conn.close()


# PRZETWARZANIE DANYCH Z FOLDERU

In [68]:
process_all_videos()


Processing A
Processing a2.mp4


I0000 00:00:1751549929.538203   38336 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1751549929.541623  103688 gl_context.cc:369] GL version: 3.2 (OpenGL ES 3.2 Mesa 25.0.7), renderer: Mesa Intel(R) UHD Graphics 620 (KBL GT2)
W0000 00:00:1751549929.572454  103681 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1751549929.588129  103684 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


Processing a3.mp4


I0000 00:00:1751549931.523569   38336 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1751549931.527720  103707 gl_context.cc:369] GL version: 3.2 (OpenGL ES 3.2 Mesa 25.0.7), renderer: Mesa Intel(R) UHD Graphics 620 (KBL GT2)
W0000 00:00:1751549931.562760  103701 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1751549931.579907  103699 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


Processing a4.mp4


...

I0000 00:00:1751550401.788787   38336 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1751550401.793258  111955 gl_context.cc:369] GL version: 3.2 (OpenGL ES 3.2 Mesa 25.0.7), renderer: Mesa Intel(R) UHD Graphics 620 (KBL GT2)
W0000 00:00:1751550401.819208  111946 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1751550401.837285  111952 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


Processing POLSKI JĘZYK MIGOWY - POLSKI ALFABET PALCOWY-Y (trimmed).mp4


I0000 00:00:1751550403.674742   38336 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1751550403.678928  111974 gl_context.cc:369] GL version: 3.2 (OpenGL ES 3.2 Mesa 25.0.7), renderer: Mesa Intel(R) UHD Graphics 620 (KBL GT2)
W0000 00:00:1751550403.706859  111966 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1751550403.766047  111966 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


Processing -Y (trimmed).mp4


I0000 00:00:1751550406.678897   38336 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1751550406.682553  111993 gl_context.cc:369] GL version: 3.2 (OpenGL ES 3.2 Mesa 25.0.7), renderer: Mesa Intel(R) UHD Graphics 620 (KBL GT2)
W0000 00:00:1751550406.708046  111985 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1751550406.726431  111984 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


# EKSPORT DO PLIKU CSV

In [73]:
import sqlite3
import csv

def export_all_tables_to_single_csv(db_file, output_csv):
    conn = sqlite3.connect(db_file)
    cursor = conn.cursor()

    # Pobierz nazwy wszystkich tabel
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tables = [row[0] for row in cursor.fetchall()]

    global_sample_id = 0
    merged_rows = []
    header = None

    for table in tables:
        cursor.execute(f"SELECT * FROM {table} ORDER BY sample_id, frame;")
        rows = cursor.fetchall()

        # Pobranie nazw kolumn (raz)
        if header is None:
            header = [description[0] for description in cursor.description]

        # Grupowanie po sample_id - 30 klatek to jedna próbka
        current_sample = None
        for row in rows:
            row = list(row)
            sample_id_local = row[header.index('sample_id')]
            frame = row[header.index('frame')]

            # Jeśli sample_id się zmienia, zwiększamy globalny licznik próbek
            if sample_id_local != current_sample:
                current_sample = sample_id_local
                global_sample_id += 1

            # Zamień lokalny sample_id na globalny
            row[header.index('sample_id')] = global_sample_id

            merged_rows.append(row)

    # Zapis do CSV
    with open(output_csv, mode='w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(header)
        writer.writerows(merged_rows)

    conn.close()
    print(f"All tables merged and exported to {output_csv}")

# Przykład użycia:
export_all_tables_to_single_csv("video_dataset.db", "videos_dataset.csv")


All tables merged and exported to merged_dataset.csv
